In [ ]:
import json
import requests

DOMAIN = 'https://adb-XXXXXX.azuredatabricks.net'
TOKEN = "XXXXXXXX"
BASE_URL = '{0}/api/2.1/'.format(DOMAIN)


In [ ]:
# https://docs.databricks.com/api/workspace/catalogs/list


def list_catalogs(full=False):
    action = "unity-catalog/catalogs"

    response = requests.get(
    BASE_URL + action,
        headers={'Authorization': 'Bearer %s' % TOKEN }
        #json=body
        )
    catalogs = response.json()["catalogs"]
    if full:
        return catalogs
    return [cat["name"] for cat in catalogs]


In [ ]:
catalogs = list_catalogs()
catalogs

In [ ]:
# https://docs.databricks.com/api/workspace/schemas/list


def list_schemas(catalog, full=False):
    action = "unity-catalog/schemas?catalog_name={0}".format(catalog)

    response = requests.get(
    BASE_URL + action,
        headers={'Authorization': 'Bearer %s' % TOKEN }
        #json=body
        )

    schemas = response.json()["schemas"]
    if full:
        return schemas
    return [schema["name"] for schema in schemas]

In [ ]:
# for cat in catalogs:
#     print(list_schemas(cat))

cat = "healthcare"
schemas = list_schemas(cat)
schemas

In [ ]:
schema = "clinicaltrialssilver"

In [ ]:
# https://docs.databricks.com/api/workspace/tables/list


def list_tables(catalog,schema, full=False):
    action = "unity-catalog/tables?catalog_name={0}&schema_name={1}&include_delta_metadata=True".format(catalog,schema)

    response = requests.get(
    BASE_URL + action,
        headers={'Authorization': 'Bearer %s' % TOKEN }
        #json=body
        )
    tables = response.json()["tables"]
    if full:
        return tables
    return [table["name"] for table in tables]

In [ ]:
tables = list_tables(cat, schema, full=False)
tables

In [ ]:
def gettablestoragelocation(catalog,schema, table, full=False):
    action = "unity-catalog/tables/{0}.{1}.{2}".format(catalog,schema,table)

    response = requests.get(
    BASE_URL + action,
        headers={'Authorization': 'Bearer %s' % TOKEN }
        #json=body
        )
    tabledetails = response.json()
    sto_loc = tabledetails["storage_location"]
    splitted = sto_loc.split("@")
    container_name = splitted[0][8:]
    path_splitted = splitted[1].split("/")
    new_path = [path_splitted[0], container_name] + path_splitted[1:]
    https_path = "https://" + "/".join(new_path) 
    print(table)
    print(sto_loc)
    print(https_path + "\n")

for table in tables:
    gettablestoragelocation(cat, schema, table)

In [ ]:
table = tables[0]
table

In [ ]:
# https://docs.databricks.com/api/workspace/grants/get


def get_data_permissions(cat, schema=None, table=None, effective_permissions=False):
    if not schema:
        full_name = cat
        securabletype = "catalog"
    elif not table:
        securabletype = "schema"
        full_name = "{0}.{1}".format(cat, schema)
    else:
        securabletype = "table"
        full_name = "{0}.{1}.{2}".format(cat, schema, table)

    if effective_permissions:
        action = "unity-catalog/effective-permissions/{0}/{1}".format(securabletype, full_name)
    else:
        action = "unity-catalog/permissions/{0}/{1}".format(securabletype, full_name)

    response = requests.get(
    BASE_URL + action,
        headers={'Authorization': 'Bearer %s' % TOKEN }
        #json=body
        )

    resp = response.json()
    if "privilege_assignments" in resp:
        return resp["privilege_assignments"]
    return resp


In [ ]:
get_data_permissions(cat, schema, table, effective_permissions=True)